In [8]:
import librosa
import numpy as np
import json

def extract_features(audio_path):
    # Load the audio file
    y, sr = librosa.load(audio_path, sr=None)
    
    # Define the hop length (number of samples per time-step)
    hop_length = int(0.050 * sr)  # 50ms window
    
    # Apply STFT
    D = np.abs(librosa.stft(y, hop_length=hop_length))
    
    # Frequency bins
    freqs = librosa.fft_frequencies(sr=sr)
    
    # Define frequency ranges
    ranges = {
        "sub_bass": (20, 60),
        "bass": (60, 250),
        "lower_midrange": (250, 500),
        "midrange": (500, 2000),
        "higher_midrange": (2000, 4000),
        "presence": (4000, 6000),
        "brilliance": (6000, 20000),
    }
    
    # Calculate the time for each column in D
    times = librosa.frames_to_time(range(D.shape[1]), sr=sr, hop_length=hop_length)
    
    # Detect note onsets
    onset_frames = librosa.onset.onset_detect(y=y, sr=sr, hop_length=hop_length)
    onset_times = librosa.frames_to_time(onset_frames, sr=sr, hop_length=hop_length)
    
    # Initialize the output dictionary
    output = []
    
    # Extract energy in the specified frequency ranges for each time-step
    for i, time in enumerate(times):
        features = {"time": float(time)}  # Convert to Python float
        for name, (low, high) in ranges.items():
            # Find the indices of the frequency bins that fall into the current range
            indices = np.where((freqs >= low) & (freqs <= high))[0]
            # Calculate the average energy in this frequency range
            energy = np.mean(D[indices, i])
            features[name] = float(energy)  # Convert to Python float
        
        # Check if there is a note onset at this time-step
            features["note_onset"] = int(time in onset_times) # Cast boolean to integer

        
        output.append(features)
    
    return json.dumps(output, indent=4)

filename = "canon"
# Use the function with your audio path
audio_path = f"/Users/garfieldgreglim/Documents/Repos/HEAD/rehab_app/rehab_flutter/assets/audio/{filename}.mp3"
# Save or print the JSON output
file_path = f"/Users/garfieldgreglim/Documents/Repos/HEAD/rehab_app/rehab_flutter/assets/data/{filename}.json"

# Save the dictionary as JSON
features_json = extract_features(audio_path)
# Save the JSON string to the file
with open(file_path, "w") as json_file:
    json_file.write(features_json) 


print("JSON file saved successfully at:", file_path)

JSON file saved successfully at: /Users/garfieldgreglim/Documents/Repos/HEAD/rehab_app/rehab_flutter/assets/data/canon.json


In [9]:
import json
import numpy as np

def normalize_and_adjust(data):
    # Adjust values lower than 0.01 to 0
    # Adjust values lower than 0.01 to 0
    for entry in data:
        for key, value in entry.items():
            if key not in ['time', 'note_onset']:  # Apply adjustment to relevant fields only
                if value < 0.01:
                    entry[key] = 0.0


    # Initialize a dictionary to store min and max values for each key
    min_max_values = {}

    # Find the min and max for each field, after adjustment
    for entry in data:
        for key, value in entry.items():
            if key not in ['time', 'note_onset']:
                if key in min_max_values:
                    min_max_values[key]['min'] = min(min_max_values[key]['min'], value)
                    min_max_values[key]['max'] = max(min_max_values[key]['max'], value)
                else:
                    min_max_values[key] = {'min': value, 'max': value}

    # Normalize the values and cut to the hundredths digit
    for entry in data:
        for key, value in entry.items():
            if key not in ['time', 'note_onset']:
                min_val = min_max_values[key]['min']
                max_val = min_max_values[key]['max']
                # Avoid division by zero if min and max are the same
                if max_val - min_val != 0:
                    normalized_value = (value - min_val) / (max_val - min_val)
                    # Round to the hundredths place
                    entry[key] = round(normalized_value, 2)
                else:
                    entry[key] = 0.0

    return data



try:
    with open(file_path, "r") as json_file:
        data = json.load(json_file)
    normalized_data = normalize_and_adjust(data)
    # Write the normalized data back to the file
    with open(file_path, "w") as json_file:
        json.dump(normalized_data, json_file, indent=4)
    print("Normalized data written back to the file.")
except FileNotFoundError:
    print(f"File not found: {file_path}")
except json.JSONDecodeError:
    print(f"File contains invalid JSON: {file_path}")

Normalized data written back to the file.


In [10]:
import json
from statistics import mode, StatisticsError, mean, median
from collections import Counter

def analyze_data(file_path):
    with open(file_path, "r") as json_file:
        data = json.load(json_file)
    
    # Initialize dictionaries for various statistics
    unique_values = {key: set() for key in data[0].keys()}
    modes = {}
    minimums = {key: float('inf') for key in data[0].keys()}
    maximums = {key: float('-inf') for key in data[0].keys()}
    averages = {key: 0 for key in data[0].keys()}
    medians = {key: 0 for key in data[0].keys()}
    unique_counts = {key: 0 for key in data[0].keys()}  # Dictionary for unique value counts

    # Populate the unique values sets and update min/max
    for entry in data:
        for key, value in entry.items():
            unique_values[key].add(value)
            minimums[key] = min(minimums[key], value)
            maximums[key] = max(maximums[key], value)
    
    # Calculate mode, average, and median for each field
    for key in unique_values.keys():
        values_list = [entry[key] for entry in data if key in entry]  # Ensure key exists in entry
        non_zero_values = [value for value in values_list if value != 0]  # Filter out zero values
        values_counter = Counter(non_zero_values)
        most_common_non_zero = values_counter.most_common(1)[0] if values_counter else (None, 0)
        modes[key] = {"mode": most_common_non_zero[0], "count": most_common_non_zero[1]}
        averages[key] = mean(non_zero_values) if non_zero_values else 0
        medians[key] = median(non_zero_values) if non_zero_values else 0
        unique_counts[key] = len(unique_values[key])  # Count of unique values
    
    # Convert sets to lists for easier JSON serialization
    for key in unique_values:
        unique_values[key] = list(unique_values[key])
    
    return unique_values, unique_counts, modes, minimums, maximums, averages, medians


unique_values, unique_counts, modes, minimums, maximums, averages, medians = analyze_data(file_path)
print("Unique Value Counts:", json.dumps(unique_counts, indent=4))
print("Modes (with count of occurrence for non-zero values):", json.dumps(modes, indent=4))
print("Minimums:", json.dumps(minimums, indent=4))
print("Maximums:", json.dumps(maximums, indent=4))
print("Averages:", json.dumps(averages, indent=4))
print("Medians:", json.dumps(medians, indent=4))

Unique Value Counts: {
    "time": 3412,
    "sub_bass": 71,
    "note_onset": 2,
    "bass": 94,
    "lower_midrange": 85,
    "midrange": 80,
    "higher_midrange": 78,
    "presence": 72,
    "brilliance": 2
}
Modes (with count of occurrence for non-zero values): {
    "time": {
        "mode": 0.05,
        "count": 1
    },
    "sub_bass": {
        "mode": 0.01,
        "count": 403
    },
    "note_onset": {
        "mode": 1,
        "count": 407
    },
    "bass": {
        "mode": 0.14,
        "count": 125
    },
    "lower_midrange": {
        "mode": 0.08,
        "count": 128
    },
    "midrange": {
        "mode": 0.02,
        "count": 176
    },
    "higher_midrange": {
        "mode": 0.01,
        "count": 227
    },
    "presence": {
        "mode": 0.06,
        "count": 106
    },
    "brilliance": {
        "mode": 1.0,
        "count": 1
    }
}
Minimums: {
    "time": 0.0,
    "sub_bass": 0.0,
    "note_onset": 0,
    "bass": 0.0,
    "lower_midrange": 0.0,
  